<a href="https://colab.research.google.com/github/HB5101/Dockship-Satellite_Image_Classification/blob/main/Satellite_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip "/content/drive/MyDrive/Attentive_AI/attentive_ai_internship_hiring_challenge-dataset.zip"

Archive:  /content/drive/MyDrive/Attentive_AI/attentive_ai_internship_hiring_challenge-dataset.zip
replace merged_data/train/2.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import LSTM, Dropout, Conv2D, TimeDistributed, MaxPooling2D, Flatten, Activation
from keras.layers import Dense

In [ ]:
train_df = pd.read_csv("/content/merged_data/train_challenge.csv")
test_df = pd.read_csv("/content/merged_data/test_challenge.csv")

In [ ]:
train_df

,0,1
0,0.jpg,Adhered
1,1.jpg,Adhered
2,2.jpg,Concrete
3,3.jpg,Concrete
4,4.jpg,Plastic & fabric
...,...,...
8639,8639.jpg,Adhered
8640,8640.jpg,Adhered
8641,8641.jpg,Adhered
8642,8642.jpg,Adhered


In [ ]:
test_df

,0,1
0,0.jpg,Adhered
1,1.jpg,Adhered
2,2.jpg,Adhered
3,3.jpg,Adhered
4,4.jpg,Adhered
...,...,...
591,591.jpg,Adhered
592,592.jpg,Adhered
593,593.jpg,Adhered
594,594.jpg,Adhered


In [ ]:
train_df['1'].value_counts()

Steel               2322
Adhered             2321
Concrete            1146
Shingle             1072
Plastic & fabric    1068
Ballasted            715
Name: 1, dtype: int64

In [ ]:
test_df['1'].value_counts()

Adhered    596
Name: 1, dtype: int64

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread("/content/merged_data/test/56.jpg")
print(img.shape)
cv2_imshow( img)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, x_valid = train_test_split( train_df, test_size=0.1, random_state=42, stratify = train_df["1"])

In [ ]:
from keras_preprocessing.image import ImageDataGenerator
DataGen = ImageDataGenerator( rescale=1./255., shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_generator=DataGen.flow_from_dataframe(dataframe=X_train, 
                                            directory='/content/merged_data/train', 
                                            x_col="0", y_col="1" ,
                                            class_mode="categorical",
                                            target_size=(300,300), batch_size=16)

valid_generator=DataGen.flow_from_dataframe(dataframe=x_valid, 
                                            directory='/content/merged_data/train', 
                                            x_col="0", y_col="1" ,
                                            class_mode="categorical", 
                                            target_size=(300,300), batch_size=16)

Found 7779 validated image filenames belonging to 6 classes.
Found 865 validated image filenames belonging to 6 classes.


In [ ]:
from keras.applications import InceptionResNetV2
base_model = InceptionResNetV2(include_top = True, weights = 'imagenet', input_shape = (299,299,3))

225214464/225209952 [==============================] - 2s 0us/step


In [ ]:
!pip install -U efficientnet

     |████████████████████████████████| 51kB 4.8MB/s 


In [ ]:
import efficientnet.keras as efn
base_model = efn.EfficientNetB3(include_top = True, weights = 'imagenet', input_shape = (300,300,3))

50094080/50086816 [==============================] - 1s 0us/step


In [ ]:
x = base_model.layers[-2].output
#x= Dense(252, activation='relu')(x)
#x= Dense(126, activation='relu')(x)
x= Dense(6, activation='softmax')(x)
model3 = Model(inputs = base_model.input, outputs =x)

In [ ]:
model2.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 260, 260, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 130, 130, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 130, 130, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 130, 130, 32) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [ ]:
for layer in model.layers[:154]:
    layer.trainable = False

In [ ]:
model3.compile(keras.optimizers.Adam(lr=0.0001),
loss="categorical_crossentropy", metrics=["accuracy"])
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [ ]:
from keras.callbacks import ModelCheckpoint
model_save_name = 'checkpoint_Eff4.h5'
path = F"/content/drive/MyDrive/Attentive_AI/{model_save_name}" 
cp_callback = ModelCheckpoint(filepath=path, monitor='val_accuracy',
                              verbose=1, save_weights_only=False ,period=1,
                              save_best_only=True, mode ='max')

In [ ]:
from sklearn.utils import class_weight
test_keys = [0,1,2,3,4,5]
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_df["1"]),
                                                 train_df["1"])
Class_dict = {test_keys[i]: class_weights[i] for i in range(len(test_keys))}


Ensemble learning pakke se try karna

In [ ]:
history = model3.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10, callbacks=[cp_callback])

In [ ]:
model2 = keras.models.load_model(path)

In [ ]:
#modelA = keras.models.load_model("/content/drive/MyDrive/Attentive_AI/checkpoint_InRe1.h5")
modelB = keras.models.load_model("/content/drive/MyDrive/Attentive_AI/checkpoint_InRe2.h5")
modelC = keras.models.load_model("/content/drive/MyDrive/Attentive_AI/checkpoint_InRe3.h5")
modelD = keras.models.load_model("/content/drive/MyDrive/Attentive_AI/checkpoint_InRe4.h5")
#modelE = keras.models.load_model("/content/drive/MyDrive/Attentive_AI/checkpoint_InRe5.h5")


In [ ]:
modelF = keras.models.load_model("/content/drive/MyDrive/Attentive_AI/checkpoint_mo1.h5")

In [ ]:
modeli = keras.models.load_model("/content/drive/MyDrive/Attentive_AI/checkpoint_Eff1.h5")

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test_df,
directory="/content/merged_data/test",
x_col="0",
y_col=None,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(299,299))

Found 596 validated image filenames.


In [ ]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
predj=model.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)

596/596 [==============================] - 12s 21ms/step


In [ ]:
predk=model1.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)

596/596 [==============================] - 14s 21ms/step


In [ ]:
predl = model2.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)

596/596 [==============================] - 14s 21ms/step


In [ ]:
pred=predl

In [ ]:
#predA =  modelA.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)
predB =  modelB.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)
predC =  modelC.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)
predD =  modelD.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)
#predE =  modelE.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)

596/596 [==============================] - 23s 35ms/step


In [ ]:
predi = modeli.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)

596/596 [==============================] - 12s 19ms/step


In [ ]:
pred2 = ( predA+predB+predC+predD+predE)/5

In [ ]:
pred = (predi+predj+predk +predl+predA +predB +predC +predD +predE)/9

In [ ]:
import numpy as np
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
labels

{0: 'Adhered',
 1: 'Ballasted',
 2: 'Concrete',
 3: 'Plastic & fabric',
 4: 'Shingle',
 5: 'Steel'}

In [ ]:
test_df = test_df.drop(["1"], axis=1)

In [ ]:
test_df.insert(1, "1", predictions)
test_df

,0,1
0,0.jpg,Ballasted
1,1.jpg,Steel
2,2.jpg,Ballasted
3,3.jpg,Concrete
4,4.jpg,Steel
...,...,...
591,591.jpg,Plastic & fabric
592,592.jpg,Steel
593,593.jpg,Plastic & fabric
594,594.jpg,Ballasted


In [ ]:
test_df.to_csv("results_attentive_Eff3tot.csv",index=False)

In [ ]:
1- basic inception resnet v2 model
2- with class weights (acc didnt improve)
3- with freezing layers (acc didnt improve)
4- with an additional layer of 256 neurons (best accuracy)
5- 256 + 126 layer (best accuracy) 
6 - ResNet 50 is worst